# [TF] Pawpularity EfficientNetB0 + metadata ensamble 😺 🐶

![](https://www.petfinder.my/images/cuteness_meter.jpg)

# Description of the notebook:

- images and tabular data(meta data)로 모델 생성
- images에 사용된 기존 모델은 EfficientNet b0 with noisy student weights 이다.
- AdamW + cosine decay 훈련했다. 
- 앙상블(ensamble) using 5-fold CV

In [1]:
lazy_submit = False # set to False to train, set False to use the stored trained weights
COLAB = False

In [2]:
# library
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import os
import gc
import cv2
from tqdm import tqdm
import seaborn as sns
import tensorflow_addons as tfa
import math

In [3]:
print(tf. __version__)

In [4]:
# gpu, cpu option
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

In [5]:
# 코드 재생산성을 위한 seed 고정
SEED = 42
np.random.seed(SEED)
IMAGE_SIZE = (224, 224)

BATCH_SIZE = 32
EPOCHS = 20
FOLDS = 5

# colab 사용시 경로
if COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  datapath = "gdrive/MyDrive/PAWPULARITY"
else:
  datapath  = ".."
# data 경로
INPUT_DIR = os.path.join(datapath, "input")
OUTPUT_DIR = os.path.join(datapath, "working")
TRAIN_IMAGES_DIR = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'train')
TRAIN_DS = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'train.csv')
TEST_IMAGES_DIR = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'test')
TEST_DS = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'test.csv')
SUBMISSION_DS = os.path.join(INPUT_DIR, "petfinder-pawpularity-score", 'sample_submission.csv')

if lazy_submit:
    TRAINED_WEIGHTS_DIR = os.path.join(INPUT_DIR,  'weights-pawpularity')
else: 
    TRAINED_WEIGHTS_DIR = OUTPUT_DIR
    
weights =  os.path.join(INPUT_DIR, "efficientnet-noisy-student-b0", "efficientnetb0_noisy_student_notop.h5")

In [6]:
!nvidia-smi

In [7]:
train_ds = pd.read_csv(TRAIN_DS)
train_ds['Pawpularity'] = train_ds['Pawpularity']/100.0
test_ds = pd.read_csv(TEST_DS)
subm_ds = pd.read_csv(SUBMISSION_DS)
meta_cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 
             'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
train_ds.shape

# Exploratory analysis

In [8]:
grouped = train_ds.copy()
grouped["cuts"] = pd.cut(train_ds["Pawpularity"], bins = np.arange(0, 1.1, 0.1),labels= np.round(np.arange(0, 1, 0.1), 1))
grouped = grouped.drop("Id", axis = 1).groupby("cuts").mean()
fig,ax = plt.subplots(6, 2, sharex = False, sharey = True, figsize = (14, 16))
col = 0
for i in range(6):
    for j in range(2):
        sns.barplot(ax = ax[i, j], x = meta_cols[col], y = "cuts",  data = grouped[meta_cols].reset_index())
        col += 1
plt.tight_layout()
plt.show()

In [9]:
# meta data heatmap
plt.figure(figsize = (10, 8))
sns.heatmap(grouped.corr())

In [10]:
# meta data clustermap
plt.figure(figsize = (10, 8))
sns.clustermap(grouped.corr())

In [11]:
# img 전처리 및 정규화
img = keras.preprocessing.image.load_img("../input/petfinder-pawpularity-score/train/00524dbf2637a80cbc80f70d3ff59616.jpg")
img = keras.preprocessing.image.img_to_array(img) / 255.0
img.shape

In [12]:
plt.imshow(img)

## Useful functions

In [13]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, img_dir, 
                 batch_size, tab_columns,
                 id, target, is_train,
                 input_size = (224, 224),
                 shuffle = True):
        
        self.df = df.copy()
        self.img_dir = img_dir
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle    
        self.n = len(self.df)
        self.tab_columns = tab_columns
        self.target = target
        self.id = id
        self.is_train = is_train
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def __getitem__(self, index):
        this_ds = self.df.iloc[index*self.batch_size:(index+1)*self.batch_size]
        images = []
        for img_id in list(this_ds[self.id].values):
            img = cv2.imread(f"{self.img_dir}/{img_id}.jpg")
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, IMAGE_SIZE, interpolation=cv2.INTER_LINEAR)
            img = np.array(img, dtype='float32') 
            # ./ 255.0 EfficientNet models expect their inputs to be float tensors of pixels with values in the [0-255] range
            images.append(img)
        if self.is_train:
            return [np.array(images), this_ds[self.tab_columns].values],this_ds[self.target].values
        else:
            return [np.array(images), this_ds[self.tab_columns].values]

## Build the model

In [14]:
NCOL = len(meta_cols)
INPUT_SHAPE = (*IMAGE_SIZE, 3)

optimizer = tfa.optimizers.AdamW(lr=1e-3, weight_decay=1e-4)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_root_mean_squared_error",
    patience = 2,
    factor = 0.5,
    verbose = 1,
    mode = 'min'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_root_mean_squared_error', patience=3
)

class CosineAnnealingLearningRateSchedule(tf.keras.callbacks.Callback):
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()

    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = math.floor(n_epochs/n_cycles)
        cos_inner = (math.pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
        return lrate_max/2 * (math.cos(cos_inner) + 1)

    def on_epoch_begin(self, epoch, logs=None):
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        self.lrates.append(lr)

ca = CosineAnnealingLearningRateSchedule(EPOCHS, EPOCHS/5, 1e-3)

rmse = tf.keras.metrics.RootMeanSquaredError()


def get_checkpoint(idx):
    return tf.keras.callbacks.ModelCheckpoint(os.path.join(OUTPUT_DIR, f'efficientnetb0_{idx}.h5'), 
                                                    monitor='val_root_mean_squared_error', verbose = 1, 
                                                    save_best_only=True, mode='min', save_weights_only = True)


def create_model():

    CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'normal'
        }
    }

    conv_base = tf.keras.applications.efficientnet.EfficientNetB0(
        weights = weights,
        include_top = False,
        input_tensor=keras.Input(shape = INPUT_SHAPE))

    conv_base.trainable = False

    def conv2d_block(mod, filters):
        out = layers.Conv2D(filters, kernel_size  = (1, 1),
                          padding='same',
                          kernel_initializer = CONV_KERNEL_INITIALIZER)(mod)
        out = layers.BatchNormalization()(out)
        out = layers.Activation("swish")(out)
        return out

    inp = tf.keras.Input(shape=INPUT_SHAPE)
    out = conv_base(inp)
    out = conv2d_block(out, 640)
    out = conv2d_block(out, 320)
    out = conv2d_block(out, 128)
    out = layers.GlobalAveragePooling2D()(out)
    out = layers.BatchNormalization()(out)
    out = layers.Dropout(0.1)(out) 

    meta_input = keras.Input(shape = (NCOL,))
    concat = layers.Concatenate(axis = 1)([out, meta_input])
    concat = layers.Dense(128)(concat) 
    concat = layers.PReLU()(concat)
    concat = layers.Dropout(0.1)(concat) 
    concat = layers.Dense(64)(concat) 
    concat = layers.PReLU()(concat)
    concat = layers.Dropout(0.1)(concat) 
    concat = layers.Dense(1, activation = "sigmoid")(concat)
    model = tf.keras.Model([inp, meta_input], concat)

    return model
    
    
def get_model():
    model = create_model()
    model.compile(loss = "binary_crossentropy", 
        optimizer = optimizer,
        metrics=[rmse])
    return model

In [15]:
check_model = get_model()
check_model.summary()

In [16]:
# model 구성 표시
tf.keras.utils.plot_model(check_model, show_shapes=True)

## Train the model

In [17]:
kf = KFold(n_splits = FOLDS, shuffle = True, random_state = SEED)

def train():
   
    models = []
    histories = []
    for idx, (train_idx, val_idx) in enumerate(kf.split(train_ds)):
        
        print(f'\n----- RUN: {idx} ------')
        
        params = dict(batch_size=32, tab_columns=meta_cols, id="Id", target="Pawpularity", is_train = True)
        train_gen = CustomDataGen(img_dir = TRAIN_IMAGES_DIR, df=train_ds.iloc[train_idx], **params)
        val_gen = CustomDataGen(img_dir = TRAIN_IMAGES_DIR, df=train_ds.iloc[val_idx], **params)
        
        model = get_model()
        
        checkpoint = get_checkpoint(idx)

        history = model.fit(train_gen,
                            validation_data = val_gen,
                            epochs = EPOCHS, 
                            callbacks = [checkpoint, es, ca],
                            max_queue_size = 3 * BATCH_SIZE)
                            
        models.append(model)
        histories.append(history.history)
        gc.collect()
        
    return models,histories

In [18]:
if not lazy_submit:
    models,histories = train()
    evals = np.mean([np.min(histories[i]['val_root_mean_squared_error']) for i in range(FOLDS)])
    
    fig,ax = plt.subplots(3,2)
    for i in range(FOLDS):
        ax = ax.flatten()
        ax[i].plot(histories[i]['loss'], label= 'loss')
        ax[i].plot(histories[i]['val_loss'], label = 'val_loss')
        ax[i].legend()
    plt.show()
    
else:
    models = []
    evals = 0
    for idx, (train_idx, val_idx) in enumerate(kf.split(train_ds)):
        params = dict(img_dir=TRAIN_IMAGES_DIR, batch_size=BATCH_SIZE, tab_columns=meta_cols, id="Id", target="Pawpularity", is_train = True, shuffle  = False)
        val_gen = CustomDataGen(df=train_ds.iloc[val_idx], **params)
        model = get_model()
        model.load_weights(os.path.join(TRAINED_WEIGHTS_DIR, f'efficientnetb0_{idx}.h5'))
        models.append(model)
        evals += model.evaluate(val_gen, 
                                    max_queue_size = 3 * BATCH_SIZE)[1] 
    evals /= FOLDS

print("AVG RMSE:", evals)

# Submission

In [19]:
# submission file 생성
predictions = []
params = dict(img_dir = TEST_IMAGES_DIR, batch_size = BATCH_SIZE, tab_columns = meta_cols, id = "Id", target = "Pawpularity", is_train = False, shuffle = False)
test_gen = CustomDataGen(df = test_ds, **params)

for i in tqdm(range(FOLDS)):
    predictions.append(models[i].predict(test_gen))
    
subm_ds["Pawpularity"] =  100 * np.array(predictions).mean(axis = 0)
subm_ds.to_csv("submission.csv", index = False)

In [20]:
subm_ds.head()